In [986]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
import re
from itertools import combinations

In [909]:
data = pd.read_csv('movie_bd_v5.csv')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
119,tt2015381,170000000,773312399,Guardians of the Galaxy,Chris Pratt|Zoe Saldana|Dave Bautista|Vin Dies...,James Gunn,All heroes start somewhere.,"Light years from Earth, 26 years after being a...",121,Action|Science Fiction|Adventure,Marvel Studios|Moving Picture Company (MPC)|Bu...,7/30/2014,7.9,2014
982,tt1232829,42000000,201585328,21 Jump Street,Channing Tatum|Jonah Hill|Brie Larson|Dave Fra...,Phil Lord|Christopher Miller,They thought the streets were mean. Then they ...,"In high school, Schmidt was a dork and Jenko w...",109,Action|Comedy|Crime,Columbia Pictures|Original Film|Stephen J. Can...,3/12/2012,6.7,2012
71,tt3488710,35000000,61181942,The Walk,Joseph Gordon-Levitt|Ben Kingsley|Charlotte Le...,Robert Zemeckis,Dream High.,The story of French high-wire artist Philippe ...,123,Adventure|Drama|Thriller,TriStar Pictures|Sony Pictures Entertainment|I...,9/30/2015,6.8,2015
1563,tt0338013,20000000,72258126,Eternal Sunshine of the Spotless Mind,Jim Carrey|Kate Winslet|Elijah Wood|Mark Ruffa...,Michel Gondry,I already forget how I used to feel about you.,"Joel Barish, heartbroken that his girlfriend u...",108,Science Fiction|Drama|Romance,Anonymous Content|This Is That Productions|Foc...,3/19/2004,7.8,2004
763,tt1189340,40000000,85412898,The Lincoln Lawyer,Matthew McConaughey|Marisa Tomei|Ryan Phillipp...,Brad Furman,This Case is a Dangerous Game of Life and Death,A lawyer conducts business from the back of hi...,119,Crime|Drama|Thriller,Lakeshore Entertainment|Sidney Kimmel Entertai...,3/17/2011,6.9,2011


In [910]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


# Предобработка

In [911]:
answers = {} # создадим словарь для ответов

# тут другие ваши предобработки колонок например:

#the time given in the dataset is in string format.
#So we need to change this in datetime format
# ...


def year_detect(release_date):
    result = re.search(r'\d{4}', release_date)
    if result:
        return int(result.group(0))
    else:
        return 1970


def month_detect(release_date):
    result = re.search(r'^\d\d?', release_date)
    if result:
        month = int(result.group(0))
        if month < 1 or month > 12:
            month = 0
        return month
    else:
        return 0


def series_aggregation_first_column (incoming_data, agg_type='sum', top=5, sort_direction=False,):
    
    result_series = pd.Series(dtype=object)

    for column1, column2 in incoming_data.items():
        
        if agg_type == 'sum':
            agg_parametr = column2
        else:
            agg_parametr = 1
            
        for item in column1.split('|'):
            
            if item in result_series.index:
                result_series[item] += agg_parametr
            else:
                result_series[item] = agg_parametr

    return result_series.sort_values(ascending=sort_direction).head(top)
  




data['profit'] = data.revenue - data.budget

#data['release_year'] = data.release_date.apply(lambda x: str(x)[-4:])

data['release_year'] = data.release_date.apply(year_detect)

data['release_month'] = data.release_date.apply(month_detect)

data['word_count_overiew'] = data.overview.apply(lambda s: len(re.findall(r'\w+', s.lower())))


In [912]:
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,release_month,word_count_overiew
1172,tt0290334,110000000,214948780,X2,Patrick Stewart|Hugh Jackman|Ian McKellen|Hall...,Bryan Singer,The time has come for those who are different ...,Professor Charles Xavier and his team of genet...,133,Adventure|Action|Science Fiction|Thriller,Twentieth Century Fox Film Corporation|Donners...,4/24/2003,6.7,2003,104948780,4,56
367,tt1314228,58000000,85280250,Did You Hear About The Morgans?,Hugh Grant|Sarah Jessica Parker|Mary Steenburg...,Marc Lawrence,We're not in Manhattan anymore.,"In New York City, an estranged couple who witn...",103,Comedy,Columbia Pictures|Castle Rock Entertainment|Re...,12/17/2009,5.0,2009,27280250,12,24
209,tt2345737,12250000,2295423,The Rover,Guy Pearce|Robert Pattinson|Scoot McNairy|Nash...,David MichÃ´d,Fear the man with nothing left to lose.,A decade after the collapse of the western wor...,100,Crime|Drama,Porchlight Films|FilmNation Entertainment|Scre...,5/18/2014,6.0,2014,-9954577,5,37
1127,tt0338094,90000000,182290266,The Haunted Mansion,Eddie Murphy|Terence Stamp|Nathaniel Parker|Ma...,Rob Minkoff,Check your pulse at the door... if you have one.,Workaholic Jim Evers and his wife/business par...,99,Thriller|Fantasy|Comedy|Family|Mystery,Walt Disney Pictures|Gunn Films|Doom Buggy Pro...,11/25/2003,5.1,2003,92290266,11,54
747,tt0480687,36000000,83160734,Hall Pass,Owen Wilson|Jason Sudeikis|Jenna Fischer|Chris...,Bobby Farrelly|Peter Farrelly,"One Week, No Rules",When best buds Rick and Fred begin to show sig...,105,Comedy,New Line Cinema|Conundrum Entertainment,2/25/2011,5.4,2011,47160734,2,71


# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [913]:
# в словарь вставляем номер вопроса и ваш ответ на него
# Пример: 
answers['1'] = 'Pirates of the Caribbean: On Stranger Tides (tt1298650)'
# запишите свой вариант ответа


In [914]:
# тут пишем ваш код для решения данного вопроса:

movie_name = data[data.budget == data.budget.max()].original_title.max()
movie_id = data[data.original_title == movie_name].imdb_id.max()

print(f"{movie_name} ({movie_id})")


Pirates of the Caribbean: On Stranger Tides (tt1298650)


In [915]:
# +
answers['1'] = 'Pirates of the Caribbean: On Stranger Tides (tt1298650)'

ВАРИАНТ 2

In [916]:
# можно добавлять разные варианты решения

# 2. Какой из фильмов самый длительный (в минутах)?

In [917]:
# думаю логику работы с этим словарем вы уже поняли, 
# по этому не буду больше его дублировать


In [918]:

movie_name = data[data.runtime == data.runtime.max()].original_title.max()

movie_id = data[data.original_title == movie_name].imdb_id.max()

print(f"{movie_name} ({movie_id})")

Gods and Generals (tt0279111)


In [919]:
# +
answers['2'] = 'Gods and Generals (tt0279111)'

# 3. Какой из фильмов самый короткий (в минутах)?





In [920]:
movie_name = data[data.runtime == data.runtime.min()].original_title.max()

movie_id = data[data.original_title == movie_name].imdb_id.max()

print(f"{movie_name} ({movie_id})")

Winnie the Pooh (tt1449283)


In [921]:
# +
answers['3'] = 'Winnie the Pooh (tt1449283)'

# 4. Какова средняя длительность фильмов?


In [922]:
round(data.runtime.mean())

110

In [923]:
# +
answers['4'] = 110

# 5. Каково медианное значение длительности фильмов? 

In [924]:
round(data.runtime.median())

107

In [925]:
# +
answers['5'] = 107

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [926]:
# лучше код получения столбца profit вынести в Предобработку что в начале

movie_name = data[data.profit == data.profit.max()].original_title.max()

movie_id = data[data.original_title == movie_name].imdb_id.max()

print(f"{movie_name} ({movie_id})")

Avatar (tt0499549)


In [927]:
answers['6'] = 'Avatar (tt0499549)'

# 7. Какой фильм самый убыточный? 

In [928]:
movie_name = data[data.profit == data.profit.min()].original_title.max()

movie_id = data[data.original_title == movie_name].imdb_id.max()

print(f"{movie_name} ({movie_id})")

The Lone Ranger (tt1210819)


In [929]:
answers['7'] = 'The Lone Ranger (tt1210819)'

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [930]:
len(data[data.profit > 0])

1478

In [931]:
answers['8'] = 1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [932]:
movie_name = data[data.profit == data[data.release_date.str.contains('2008', na='')].profit.max()].original_title.max()

movie_id = data[data.original_title == movie_name].imdb_id.max()

print(f"{movie_name} ({movie_id})")

The Dark Knight (tt0468569)


In [933]:
answers['9'] = 'The Dark Knight (tt0468569)'

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [934]:
movie_name = data[data.profit == data.query('2012 <= release_year <= 2014').profit.min()].original_title.max()

movie_id = data[data.original_title == movie_name].imdb_id.max()

print(f"{movie_name} ({movie_id})")

The Lone Ranger (tt1210819)


In [935]:
answers['10'] = 'The Lone Ranger (tt1210819)'

In [936]:
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals (tt0279111)',
 '3': 'Winnie the Pooh (tt1449283)',
 '4': 110,
 '5': 107,
 '6': 'Avatar (tt0499549)',
 '7': 'The Lone Ranger (tt1210819)',
 '8': 1478,
 '9': 'The Dark Knight (tt0468569)',
 '10': 'The Lone Ranger (tt1210819)'}

# 11. Какого жанра фильмов больше всего?

In [937]:
# эту задачу тоже можно решать разными подходами, попробуй реализовать разные варианты
# если будешь добавлять функцию - выноси ее в предобработку что в начале

genre_series = pd.Series(data=[0], index=['none'])

for genre_all, genre_count in data.genres.value_counts().items():
    for genre in genre_all.split('|'):
        if genre in genre_series.index:
            genre_series[genre] += genre_count
        else:
            genre_series[genre] = genre_count

genre_series.sort_values(ascending=False).head(5)


Drama        782
Comedy       683
Thriller     596
Action       582
Adventure    415
dtype: int64

In [938]:
answers['11'] = 'Drama'

ВАРИАНТ 2

можно сделать чере словарь..

In [939]:
genre_counter = Counter()

for genre_all, genre_count in data.genres.value_counts().items():
    for genre in genre_all.split('|'):
        genre_counter[genre] += genre_count

genre_counter
# TODO - поиск максимального значения


Counter({'Comedy': 683,
         'Drama': 782,
         'Romance': 308,
         'Horror': 176,
         'Thriller': 596,
         'Action': 582,
         'Crime': 315,
         'Family': 260,
         'Animation': 139,
         'History': 62,
         'Science Fiction': 248,
         'Adventure': 415,
         'Fantasy': 222,
         'Mystery': 168,
         'Music': 64,
         'War': 58,
         'Documentary': 8,
         'Western': 19,
         'Foreign': 2})

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [940]:
data.groupby(['genres'])['profit'].sum()

genres
Action                                                 1108780815
Action|Adventure                                        134679452
Action|Adventure|Animation|Family|Fantasy               424987477
Action|Adventure|Animation|Family|Science Fiction       539442092
Action|Adventure|Animation|Science Fiction|Thriller      18428063
                                                          ...    
Western|Adventure                                         8103450
Western|Animation|Adventure|Comedy|Family                42563539
Western|Animation|Family|Comedy|Music                    -6048539
Western|Drama|Adventure|Thriller                        397950503
Western|History|War                                    -119180039
Name: profit, Length: 652, dtype: int64

In [941]:
genre_profit = pd.Series(data=[0], index=['none'])

for genre_all, profit in data.groupby(['genres'])['profit'].sum().items():
    for genre in genre_all.split('|'):
        if genre in genre_profit.index:
            genre_profit[genre] += profit
        else:
            genre_profit[genre] = profit

genre_profit.sort_values(ascending=False).head(3)



Adventure    86070466981
Action       80187900065
Comedy       59774752721
dtype: int64

In [942]:
answers['12'] = 'Adventure'

# 13. У какого режиссера самые большие суммарные кассовые сбооры?

In [943]:
director_revenue = pd.Series(data=[0], index=['none'])

for director_all, revenue in data.groupby(['director'])['revenue'].sum().items():
    for director in director_all.split('|'):
        if director in director_revenue.index:
            director_revenue[director] += revenue
        else:
            director_revenue[director] = revenue

director_revenue.sort_values(ascending=False).head(3)

Peter Jackson        6490593685
Christopher Nolan    4167548502
David Yates          4154295625
dtype: int64

In [944]:
answers['13'] = 'Peter Jackson'

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [945]:

director_genres = data[ ['director','genres'] ][data.genres.str.contains('Action', na=False)]

director_genres = director_genres.groupby(['director'])['genres'].count()

series_aggregation_first_column(director_genres, 'sum', top=5)


Robert Rodriguez      9
Paul W.S. Anderson    7
Michael Bay           7
Ridley Scott          6
Antoine Fuqua         6
dtype: int64

In [946]:
answers['14'] = 'Robert Rodriguez'

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [947]:

top_actor_2012 = data.query('release_year == 2012').groupby(['cast'])['revenue'].sum()

series_aggregation_first_column( top_actor_2012 ,'sum', top=1)


Chris Hemsworth    2027450773
dtype: int64

In [948]:
answers['15'] = 'Chris Hemsworth'

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [949]:
# высокобюджетные - фильмы, где бюджет выше среднего по данной выборке

top_actor = data[data.budget > data.budget.mean()].groupby(['cast'])['revenue'].count()

series_aggregation_first_column( top_actor ,'sum', top=6)


Matt Damon           18
Adam Sandler         17
Angelina Jolie       16
Samuel L. Jackson    15
Eddie Murphy         15
Tom Cruise           15
dtype: int64

In [950]:
answers['16'] = 'Matt Damon'

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [951]:
favorite_genre = data[data.cast.str.contains('Nicolas Cage', na=False)].groupby(['genres'])['cast'].count()

series_aggregation_first_column( favorite_genre ,'sum', top=5)

Action      17
Thriller    15
Drama       12
Crime       10
Fantasy      8
dtype: int64

In [952]:
answers['17'] = 'Action'

# 18. Самый убыточный фильм от Paramount Pictures

In [953]:
movie_name = data[data.profit == data[data.production_companies.str.contains('Paramount Pictures')].profit.min()].original_title.max()

movie_id = data[data.original_title == movie_name].imdb_id.max()

print(f"{movie_name} ({movie_id})")



K-19: The Widowmaker (tt0267626)


In [954]:
answers['18'] = 'K-19: The Widowmaker (tt0267626)'

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [955]:
data.groupby(['release_year'])['revenue'].sum().sort_values(ascending=False).head(1)

release_year
2015    25449202382
Name: revenue, dtype: int64

In [956]:
answers['19'] = 2015

# 20. Какой самый прибыльный год для студии Warner Bros?

In [957]:
wb_profit = data[data.production_companies.str.contains('Warner Bros', na=False)]

wb_profit.groupby(['release_year'])['profit'].sum().sort_values(ascending=False).head(1)

release_year
2014    2295464519
Name: profit, dtype: int64

In [958]:
# series_aggregation_first_column( _ ,'sum', top=6)
answers['20'] = 2014

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [959]:
# Вариант 1
data.release_month.value_counts().sort_values().tail(1)

9    227
Name: release_month, dtype: int64

In [960]:
# Вариант 2
data.groupby(['release_month'])['imdb_id'].count().sort_values().tail(1)

release_month
9    227
Name: imdb_id, dtype: int64

In [961]:
# series_aggregation_first_column( _ ,'sum', top=6)
answers['21'] = 'Сентябрь'

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [962]:
# data.query('6<=release_month<=8').shape
# len(data.query('6 <= release_month <= 8'))

len(data.query('release_month in [6, 7, 8]'))

450

In [963]:
# series_aggregation_first_column( _ ,'sum', top=6)
answers['22'] = 450

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [964]:

winter_director = data.query('release_month in [1, 2, 12]').director.value_counts()

series_aggregation_first_column(winter_director,'sum', top=3)

Peter Jackson        7
Steven Soderbergh    6
Clint Eastwood       6
dtype: int64

In [965]:
# series_aggregation_first_column( _ ,'sum', top=6)
answers['23'] = 'Peter Jackson'

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [966]:

max_len = data.original_title.str.len().max()

data[data.original_title.str.len() == max_len].production_companies.iloc[0]



'Twentieth Century Fox Film Corporation|Four By Two Productions'

In [967]:
# series_aggregation_first_column( _ ,'sum', top=6)
answers['24'] = 'Four By Two Productions' 

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [968]:
# production_companies_result = pd.DataFrame(columns=['production_companies', 'overview_word_count'])

# prod_companies = data[['production_companies', 'overview']].head(50)

# for i in prod_companies.index:
#     companies = prod_companies.iloc[i].production_companies
#     overview = prod_companies.iloc[i].overview
#     word_count = len(re.findall(r'\w+', overview.lower()))
    
#     for prod_companie in companies.split('|'):
        
#         new_location = len(production_companies_result)
        
#         production_companies_result.loc[new_location] = [prod_companie, word_count]


# production_companies_result.groupby(['production_companies'])['overview_word_count'].max()

# production_companies_result.production_companies.value_counts()
# production_companies_result.overview_word_count.max()

In [969]:
# data['word_count_overiew']

word_count_overiew_max = data[data.word_count_overiew > data.word_count_overiew.mean()].word_count_overiew.max()

data[data.word_count_overiew == word_count_overiew_max].production_companies.iloc[0]


'Lions Gate Films|Lakeshore Entertainment|GreeneStreet Films|Midnight Picture Show'

In [970]:
data[data.word_count_overiew == word_count_overiew_max].production_companies.iloc[0].split('|')[3]

'Midnight Picture Show'

In [971]:
# 
answers['25'] = 'Midnight Picture Show'

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [972]:

top_movies = ['Inside Out','The Dark Knight','12 Years a Slave']
#     'BloodRayne, The Adventures of Rocky & Bullwinkle',
#     'Batman Begins, The Lord of the Rings: The Return of the King, Upside Down',
#     '300, Lucky Number Slevin, Kill Bill: Vol. 1',
#     'Upside Down, Inside Out, Iron Man']



In [973]:
count_last_procent = round(len(data)*0.01)

last_procent = data.vote_average.sort_values(ascending=False).head(count_last_procent).index

data.loc[last_procent][['original_title','vote_average']]


,original_title,vote_average
599,The Dark Knight,8.1
125,The Imitation Game,8.0
9,Inside Out,8.0
118,Interstellar,8.0
34,Room,8.0
1081,The Lord of the Rings: The Return of the King,7.9
138,The Grand Budapest Hotel,7.9
1183,The Wolf of Wall Street,7.9
370,Inception,7.9
119,Guardians of the Galaxy,7.9


In [981]:
# Проверка наличия 3-х фильмов в выборке 1% лучших фильмов
data.loc[last_procent].query('original_title in @top_movies')[['original_title','vote_average']]

top_movies = ['Upside Down','Inside Out','Iron Man']
data.query('original_title in @top_movies')[['original_title','vote_average']]

,original_title,vote_average
9,Inside Out,8.0
601,Iron Man,7.3
1007,Upside Down,6.1


In [975]:
# 
answers['26'] = 'Inside Out, The Dark Knight, 12 Years a Slave'

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [995]:
a = list(combinations('carl', 3))

a


[('c', 'a', 'r'), ('c', 'a', 'l'), ('c', 'r', 'l'), ('a', 'r', 'l')]

In [1008]:
# series_aggregation_first_column( _ ,'sum', top=6)
answers['27'] = 'Daniel Radcliffe & Rupert Grint'

ВАРИАНТ 2

In [1002]:
pair_actor1 = 'Johnny Depp & Helena Bonham Carter'.split(' & ')
data[data.cast.str.contains(pair_actor1[0]) & data.cast.str.contains(pair_actor1[1])].imdb_id.count()


6

In [1004]:
pair_actor2 = 'Ben Stiller & Owen Wilson'.split(' & ')
data[data.cast.str.contains(pair_actor2[0]) & data.cast.str.contains(pair_actor2[1])].imdb_id.count()

6

In [1011]:

pair_actor1 = 'Vin Diesel & Paul Walker'.split(' & ')
data[data.cast.str.contains(pair_actor1[0]) & data.cast.str.contains(pair_actor1[1])].imdb_id.count()

5

In [1006]:
pair_actor1 = 'Adam Sandler & Kevin James'.split(' & ')
data[data.cast.str.contains(pair_actor1[0]) & data.cast.str.contains(pair_actor1[1])].imdb_id.count()

5

In [1007]:
pair_actor1 = 'Daniel Radcliffe & Rupert Grint'.split(' & ')
data[data.cast.str.contains(pair_actor1[0]) & data.cast.str.contains(pair_actor1[1])].imdb_id.count()

8

# Submission

In [1009]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals (tt0279111)',
 '3': 'Winnie the Pooh (tt1449283)',
 '4': 110,
 '5': 107,
 '6': 'Avatar (tt0499549)',
 '7': 'The Lone Ranger (tt1210819)',
 '8': 1478,
 '9': 'The Dark Knight (tt0468569)',
 '10': 'The Lone Ranger (tt1210819)',
 '11': 'Drama',
 '12': 'Adventure',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker (tt0267626)',
 '19': 2015,
 '20': 2014,
 '21': 'Сентябрь',
 '22': 450,
 '23': 'Peter Jackson',
 '24': 'Four By Two Productions',
 '25': 'Midnight Picture Show',
 '26': 'Inside Out, The Dark Knight, 12 Years a Slave',
 '27': 'Daniel Radcliffe & Rupert Grint'}

In [1010]:
# и убедиться что ни чего не пропустил)
len(answers)

27